# PoC: WeCookio multiagent Strands Agents

This notebook demonstrates the WeCookio CrewAI implementation with cost-optimized model selection using uv package manager.

### Collaborative Multi-agent withdelegation

### Installing Required Packages with uv

In [16]:
# Install required packages using uv
!uv add ipykernel strands-agents==1.2.0 strands-agents-tools boto3 botocore pyyaml ipywidgets prompt-template

Resolved 224 packages in 1ms
Audited 213 packages in 0.08ms


### Import Libraries and Load Configuration

In [17]:
import yaml
import logging
from strands import Agent
from strands.multiagent import Swarm

print("Libraries imported successfully!")

# Enable debug logs and print them to stderr
logging.getLogger("strands.multiagent").setLevel(logging.DEBUG)
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)


Libraries imported successfully!


### Loading AWS Enviroment variables

In [18]:
from dotenv import load_dotenv
import os

load_dotenv()
AWS_ACCESS_KEY_ID=os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY=os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_DEFAULT_REGION=os.getenv("AWS_REGION")

### Load Configuration Files

In [19]:
#Check base folder for relative paths
import os
print (os.getcwd())
prefix = "strands/latest/"
# Load the integration configuration
with open(prefix+'config/integration_config.yaml', 'r') as file:
    integration_config = yaml.safe_load(file)
# Load the crew configuration
with open(prefix+'config/config.yaml', 'r') as file:
    strand_config = yaml.safe_load(file)
with open(prefix+'config/agents-swarm.yaml', 'r') as file:
    agents_config = yaml.safe_load(file)
with open(prefix+'config/tasks-swarm.yaml', 'r') as file:
    tasks_config = yaml.safe_load(file)

print("Configuration files loaded successfully!")
print(f"Integration config keys: {list(integration_config.keys())}")
print(f"Crew Orchestration keys: {list(strand_config.keys())}")
print(f"Agents config keys: {list(agents_config.keys())}")
print(f"Tasks config keys: {list(tasks_config.keys())}")

/home/e2its/dev/weCookio/researches
Configuration files loaded successfully!
Integration config keys: ['aws_bedrock', 'database', 'api_endpoints']
Crew Orchestration keys: ['crew', 'agents', 'tasks', 'integration_config', 'model_assignment', 'execution', 'memory', 'input_schema', 'output_schema', 'cost_optimization', 'performance_metrics', 'quality_standards', 'monitoring', 'example_input', 'example_output']
Agents config keys: ['Culinary Coordinator', 'Dietary Compliance Specialist', 'Ingredient Substitution Expert', 'Culinary Experience Optimizer', 'Quality Assurance Chef']
Tasks config keys: ['Enhance Recipe']


###  Model Selection based on complexity

In [20]:
def select_model_for_complexity(complexity, integration_config):
    """Select the appropriate model based on task complexity"""
    try:
        complexity = complexity + "_model"
        return integration_config["aws_bedrock"][complexity]
    except KeyError:
        raise ValueError(f"Invalid complexity level: {complexity}")

# Example usage
high_complexity_model = select_model_for_complexity("high_complexity", integration_config)
medium_complexity_model = select_model_for_complexity("medium_complexity", integration_config)
low_complexity_model = select_model_for_complexity("low_complexity", integration_config)
ultra_low_complexity_model = select_model_for_complexity("ultra_low_complexity", integration_config)

print(f"High complexity model: {high_complexity_model['model_id']}")
print(f"Medium complexity model: {medium_complexity_model['model_id']}")
print(f"Low complexity model: {low_complexity_model['model_id']}")
print(f"Ultra-Low complexity model: {ultra_low_complexity_model['model_id']}")

High complexity model: bedrock/amazon.nova-premier-v1:0
Medium complexity model: bedrock/amazon.nova-pro-v1:0
Low complexity model: bedrock/amazon.nova-lite-v1:0
Ultra-Low complexity model: bedrock/amazon.nova-micro-v1:0


### Create Agents with Model-Specific Configuration

In [21]:

from datetime import datetime
import boto3
from strands.models import BedrockModel
from prompt_template import PromptTemplate, InvalidTemplateKeysError, MissingTemplateValuesError, TemplateSerializationError

# Global metrics storage
execution_metrics = {
    'start_time': None,
    'total_tokens': 0,
    'total_cost': 0.0,
    'cycle_durations': [],
    'tool_usage': {},
    'errors': [],
    'agent_performance': {}
}

def safe_get_metrics(kwargs):
    """Safely extract metrics from kwargs"""
    try:
        metrics = kwargs.get('metrics')
        if metrics is None:
            return {
                'accumulated_usage': {'totalTokens': 0},
                'cycle_durations': [],
                'tool_metrics': {}
            }
        return metrics
    except Exception as e:
        print(f"⚠️  Warning: Could not extract metrics: {e}")
        return {
            'accumulated_usage': {'totalTokens': 0},
            'cycle_durations': [],
            'tool_metrics': {}
        }

def event_loop_tracker(**kwargs):
    """Enhanced event loop tracker with proper error handling and metrics collection"""
    try:
        # Track event loop lifecycle
        if kwargs.get("init_event_loop", False):
            print("🔄 Event loop initialized")
            execution_metrics['start_time'] = datetime.now()
        elif kwargs.get("start_event_loop", False):
            print("▶️ Event loop cycle starting")
        elif kwargs.get("start", False):
            print("📝 New cycle started")
        elif "message" in kwargs:
            #print(f"📬 New message created: {kwargs['message']['role']}")
            message=kwargs['message']['content'][0]
            if message.get("text"):
                print(f"📬 New message created: {message['text']}")
            else:
                print(f'''📬 New Tool Call created {message['toolResult']['status']}:{message['toolResult']['content'][0]['text']}''')
        elif kwargs.get("complete", False):
            print("✅ Cycle completed")
            # Update global metrics
            metrics = safe_get_metrics(kwargs)
            if metrics['cycle_durations']:
                execution_metrics['cycle_durations'].extend(metrics['cycle_durations'])
            if metrics['accumulated_usage']['totalTokens']:
                execution_metrics['total_tokens'] += metrics['accumulated_usage']['totalTokens']
        elif kwargs.get("force_stop", False):
            print(f"🛑 Event loop force-stopped: {kwargs.get('force_stop_reason', 'unknown reason')}")
            execution_metrics['errors'].append({
                'type': 'force_stop',
                'reason': kwargs.get('force_stop_reason', 'unknown'),
                'timestamp': datetime.now().isoformat()
            })

        # Track tool usage
        if "current_tool_use" in kwargs and kwargs["current_tool_use"].get("name"):
            tool_name = kwargs["current_tool_use"]["name"]
            print(f"🔧 Using tool: {tool_name}")
            execution_metrics['tool_usage'][tool_name] = execution_metrics['tool_usage'].get(tool_name, 0) + 1

        # Show metrics when data is available
        #if "data" in kwargs:
            # Only show first 20 chars of each chunk for demo purposes
            #data_snippet = kwargs["data"][:20] + ("..." if len(kwargs["data"]) > 20 else "")
            #print(f"📟 Text: {data_snippet}")   
    except Exception as e:
        error_msg = f"Error in event loop tracker: {e}"
        print(f"❌ {error_msg}")
        execution_metrics['errors'].append({
            'type': 'event_loop_tracker_error',
            'message': str(e),
            'timestamp': datetime.now().isoformat()
        })
    
def agent_description_composer(agent_config):
    """Compose the agent description"""
    return f"""
    Role: {agent_config.get("role", None)}
    Goal: {agent_config.get("goal", None)}
    Backstory: {agent_config.get("backstory", None)}
    """

def create_agent_with_model(agent_config, model_config, **kwargs):
    """Create an agent with specific model configuration"""
    llm = BedrockModel(
        model_id=model_config["model_id"].split("/")[-1],
        temperature=model_config.get("model_kwargs").get("temperature", 0.6),
        top_p=model_config.get("model_kwargs").get("top_p", 0.9),
        max_tokens=model_config.get("model_kwargs").get("max_tokens", 4000)
    )

    args = {}
    args["model"]=llm
    args["system_prompt"]=agent_config.get("prompt", None)
    args["name"]=agent_config.get("name", None)
    args["description"]=agent_description_composer(agent_config)
    args["callback_handler"]=event_loop_tracker
    
    if not agent_config.get("verbose",True):
        args["callback_handler"]=None
    
    return Agent(**args)

def print_agent_info(agent, model_config) -> None:
    """Print agent creation information."""
    print(
        f"Created agent: {agent.name}\n"
        f"Agent Description: {agent.description}\n"
        f"Using model: {agent.model.get_config()}"
    )
    print("\n")

In [22]:

#agent_name = "Culinary Coordinator"
agent_config = agents_config["Culinary Coordinator"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(strand_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
culinary_coordinator_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(culinary_coordinator_agent, model_config)

agent_name = "Dietary Compliance Specialist"
agent_config = agents_config["Dietary Compliance Specialist"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(strand_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
dietary_compliance_specialist_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(dietary_compliance_specialist_agent, model_config)

agent_name = "Ingredient Substitution Expert"
agent_config = agents_config["Ingredient Substitution Expert"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(strand_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
ingredient_substitution_expert_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(ingredient_substitution_expert_agent, model_config)

agent_name = "Culinary Experience Optimizer"
agent_config = agents_config["Culinary Experience Optimizer"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(strand_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
culinary_experience_optimizer_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(culinary_experience_optimizer_agent, model_config)

agent_name = "Quality Assurance Chef"
agent_config = agents_config["Quality Assurance Chef"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(strand_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
quality_assurance_chef_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(quality_assurance_chef_agent, model_config)


Created agent: Culinary Coordinator
Agent Description: 
    Role: Master Chef & Project Manager
    Goal: Orchestrate recipe adaptation ensuring maximum culinary excellence and strict dietary compliance
    Backstory: Executive chef with 20+ years experience in dietary adaptations and team coordination
    
Using model: {'model_id': 'amazon.nova-lite-v1:0', 'temperature': 0.4, 'top_p': 0.9, 'max_tokens': 2000}


Created agent: Dietary Compliance Specialist
Agent Description: 
    Role: Nutritionist & Food Safety Expert
    Goal: Ensure 100% compliance with dietary restrictions and food safety standards
    Backstory: Certified nutritionist and food safety expert with deep knowledge of allergens and dietary requirements
    
Using model: {'model_id': 'amazon.nova-micro-v1:0', 'temperature': 0.2, 'top_p': 0.9, 'max_tokens': 1000}


Created agent: Ingredient Substitution Expert
Agent Description: 
    Role: Culinary and dietary sustitutions Specialist
    Goal: Find optimal ingredient sub

### Create a Flow

In [23]:
# Example recipe for analysis
recipe_request = {
    "food_name": "Canelones de ternera XXL al Pedro Ximenez",
    "servings": 4,
    "intolerances": [
        "Milk",
        "Eggs",
        "Beef",
        "Corn",
        "Gluten",
        "Rice",
        "Garlic"
    ],
    "exclusions": ["Pepper"],
    "preferences": [],
    "output_language": "spanish",
    "country": "Spain"
}


In [24]:
## Support functions

def variable_injection(task:dict, recipe_context:dict):
    try:
        prompt = PromptTemplate(name=task.get("name"), template=f'''{task.get("description")}''').to_string(**recipe_context)
    except MissingTemplateValuesError as e:
        print(f"Missing values: {e.missing_values}")  # {'name'}
    except InvalidTemplateKeysError as e:
        print(f"Invalid keys: {e.invalid_keys}")  # {'name'}
    except TemplateSerializationError as e:
        print(f"Serialization error: {e.message}")  # 'Invalid template string'
    except Exception as e:
        print(f"Unexpected error: {e}")  # 'Unexpected error'
    return prompt
    

In [25]:
tasks = {
    "enhance_recipe": {
        "description": f'''{variable_injection(tasks_config['Enhance Recipe'], recipe_request)}''',
        "dependencies": []
    }
}
print(tasks['enhance_recipe'].get("description"))

Enhance the recipe with the following details:
## RECIPE CONTEXT
- **Recipe Name**: Canelones de ternera XXL al Pedro Ximenez
- **Servings**: 4
- **Dietary Restrictions**: ["Milk", "Eggs", "Beef", "Corn", "Gluten", "Rice", "Garlic"], ["Pepper"]
- **Preferences**: []
- **Target Country**: Spain
- **Output Language**: spanish

## TASK OBJECTIVE
Transform the requested recipe to be 100% compliant with dietary restrictions and adapted to preferences 
while maximizing culinary experience and preserving authentic flavor/texture characteristics and provide the expected output indicated bellow.

**Expected output:**
- Normalize the recipe and instructions to the target Spain
- Return the recipe and instructions in the target Spain format and language spanish
- Validation status for Quality checks


In [26]:
# Create a swarm with agents
swarm = Swarm(
    [culinary_coordinator_agent, ingredient_substitution_expert_agent, culinary_experience_optimizer_agent,dietary_compliance_specialist_agent,quality_assurance_chef_agent],
    max_handoffs=45,
    max_iterations=15,
    execution_timeout=180.0,  # 3 minutes
    node_timeout=45.0,       # 30 seconds per agent
    repetitive_handoff_detection_window=3,  # There must be >= 3 unique agents in the last 8 handoffs
    repetitive_handoff_min_unique_agents=1    
)

DEBUG | strands.multiagent.swarm | nodes=<['Culinary Coordinator', 'Ingredient Substitution Expert', 'Culinary Experience Optimizer', 'Dietary Compliance Specialist', 'Quality Assurance Chef']> | initialized swarm with nodes
DEBUG | strands.multiagent.swarm | tool_count=<1>, node_count=<5> | injected coordination tools into agents


### Run the Flow

In [27]:
try:
    # Run the swarm
    result =swarm(tasks['enhance_recipe'].get("description"))
    # Access the final result
    print(f"Status: {result.status}")
    print(f"Node history: {[node.node_id for node in result.node_history]}")    
except Exception as e:
    print(f"Error running Workflow: {e}")
    print("Make sure you have AWS credentials configured and the required permissions.")


DEBUG | strands.multiagent.swarm | starting swarm execution
DEBUG | strands.multiagent.swarm | current_node=<Culinary Coordinator> | starting swarm execution with node
DEBUG | strands.multiagent.swarm | max_handoffs=<45>, max_iterations=<15>, timeout=<180.0>s | swarm execution config
DEBUG | strands.multiagent.swarm | current_node=<Culinary Coordinator>, iteration=<1> | executing node


🔄 Event loop initialized
📝 New cycle started
▶️ Event loop cycle starting


DEBUG | strands.multiagent.swarm | from_node=<Culinary Coordinator>, to_node=<Ingredient Substitution Expert> | handed off from agent to agent


🔧 Using tool: handoff_to_agent
📬 New message created: <thinking>To enhance the recipe "Canelones de ternera XXL al Pedro Ximenez" while adhering to the dietary restrictions and preferences, I need to first understand the original recipe and then coordinate with the agents to find suitable ingredient substitutions and optimize the culinary experience. I will start by requesting the original recipe from the user and then hand off to the Ingredient Substitution Expert to find suitable replacements for the restricted ingredients. After that, I will coordinate with the Culinary Experience Optimizer to ensure the final recipe maintains its authentic flavor and texture. Finally, I will validate the recipe with the Dietary Compliance Specialist and Quality Assurance Chef to ensure it meets all requirements and standards.</thinking>

📬 New Tool Call created success:Handed off to Ingredient Substitution Expert: Please find suitable ingredient substitutions for the following restricted ingredient

DEBUG | strands.multiagent.swarm | from_node=<Ingredient Substitution Expert>, to_node=<Culinary Experience Optimizer> | handed off from agent to agent


...

📬 New message created: <thinking>The Quality Assurance Chef has been handed off the task to validate the adapted recipe 'Canelones de ternera XXL al Pedro Ximenez' to ensure it complies with all dietary restrictions, food safety standards, and quality assurance for Spain.

Once the Quality Assurance Chef completes the validation, I will receive the final confirmation and can then proceed to compile the final recipe in Spanish, ensuring it meets all the specified requirements.</thinking>

I will now wait for the Quality Assurance Chef to complete the validation and provide the final confirmation. Once received, I will compile the final recipe in Spanish, ensuring it meets all the specified requirements.
🔄 Event loop initialized
📝 New cycle started
▶️ Event loop cycle starting


DEBUG | strands.multiagent.swarm | from_node=<Quality Assurance Chef>, to_node=<Dietary Compliance Specialist> | handed off from agent to agent
DEBUG | strands.multiagent.swarm | from_node=<Dietary Compliance Specialist>, to_node=<Ingredient Substitution Expert> | handed off from agent to agent
DEBUG | strands.multiagent.swarm | from_node=<Ingredient Substitution Expert>, to_node=<Culinary Experience Optimizer> | handed off from agent to agent
DEBUG | strands.multiagent.swarm | from_node=<Culinary Experience Optimizer>, to_node=<Quality Assurance Chef> | handed off from agent to agent


🔧 Using tool: handoff_to_agent
🔧 Using tool: handoff_to_agent
🔧 Using tool: handoff_to_agent
🔧 Using tool: handoff_to_agent
📬 New message created: <thinking>To ensure the recipe complies with all dietary restrictions, food safety standards, and quality assurance for Spain, I need to validate each component of the recipe against the provided dietary restrictions and preferences. The recipe must be adapted to the Spanish context, ensuring the ingredients are available in Spain and the measurements are appropriate for the country. Additionally, I need to ensure the recipe maintains its authentic flavor and texture while being safe and clear for the end user. I will start by verifying the ingredient substitutions and ensuring they are specific and compatible with all dietary restrictions. I will then confirm the availability of these ingredients in Spain and ensure the cooking instructions are clear and safe. Finally, I will check the overall culinary experience and quality assurance frame

DEBUG | strands.multiagent.swarm | node=<Quality Assurance Chef> | node execution completed
DEBUG | strands.multiagent.swarm | node=<Quality Assurance Chef> | no handoff occurred, marking swarm as complete
DEBUG | strands.multiagent.swarm | status=<Status.COMPLETED> | swarm execution completed
DEBUG | strands.multiagent.swarm | node_history_length=<9>, time=<69.10>s | metrics


📬 New message created: <thinking>I have successfully handed off the tasks to the relevant agents for specialized assistance. Each agent is tasked with ensuring specific aspects of the recipe are compliant and optimized. I will now await their feedback and results to finalize the recipe validation and ensure it meets all the criteria outlined in the Quality Assurance Framework.</thinking>
Status: Status.COMPLETED
Node history: ['Culinary Coordinator', 'Quality Assurance Chef', 'Culinary Coordinator', 'Culinary Experience Optimizer', 'Quality Assurance Chef', 'Ingredient Substitution Expert', 'Culinary Coordinator', 'Dietary Compliance Specialist', 'Quality Assurance Chef']


In [28]:
def get_final_swarm_result(result):
    """Obtiene el resultado final del swarm de forma limpia"""
    if result.status.value == 'failed':
        print("⚠️ El swarm falló - mostrando último resultado disponible")
    
    # Obtener el último agente
    last_agent_name = result.node_history[-1].node_id
    last_result = result.results[last_agent_name]
    
    # Extraer contenido
    content = last_result.result.message['content'][0]['text']
    
    # Limpiar thinking
    if '<thinking>' in content and '</thinking>' in content:
        start = content.find('</thinking>') + len('</thinking>')
        content = content[start:].strip()
    
    return {
        'agent': last_agent_name,
        'content': content,
        'status': last_result.status,
        'execution_time': last_result.execution_time,
        'tokens_used': last_result.accumulated_usage['totalTokens']
    }

# Usar la función
final_result = get_final_swarm_result(result)
print("=== RESULTADO FINAL DEL SWARM ===")
print(f"Agente final: {final_result['agent']}")
print(f"Contenido:\n{final_result['content']}")
print(f"Tiempo de ejecución: {final_result['execution_time']}ms")
print(f"Tokens utilizados: {final_result['tokens_used']}")





=== RESULTADO FINAL DEL SWARM ===
Agente final: Quality Assurance Chef
Contenido:

Tiempo de ejecución: 6441ms
Tokens utilizados: 14875


In [29]:
# Ver resultado del agente coordinador
agent_name = 'Culinary Coordinator'
agent_result=result.results[agent_name]

print(f"\n=== RESULTADO DE {agent_name} ===")
print(agent_result.result.message['content'][0]['text'])
print(f"Status: {agent_result.status}")
print(f"Execution time: {agent_result.execution_time}ms")


=== RESULTADO DE Culinary Coordinator ===
<thinking>I have handed off the task to the Ingredient Substitution Expert, Dietary Compliance Specialist, and Quality Assurance Chef to provide ingredient substitutions, confirm dietary compliance, and validate the final recipe. I will wait for their feedback to ensure the recipe meets all quality standards and provides an exceptional culinary experience. Once I receive their confirmation, I will compile the final recipe and provide it to the user.</thinking>

**Enhanced Recipe: Canelones de ternera XXL al Pedro Ximenez**

**Ingredients:**
- 500g of turkey or chicken (substituted for beef)
- 2 large zucchinis (instead of corn)
- 2 large carrots (instead of rice)
- 1 large onion (instead of garlic)
- 1 cup of vegetable broth (instead of milk)
- 1 cup of water (instead of eggs)
- 1 cup of tomato sauce (instead of gluten-containing sauces)
- 1 cup of spinach (instead of pepper)
- 10 sheets of rice paper (instead of gluten-containing pasta)

**In

In [30]:
# Ver todos los resultados de cada agente
for agent_name, agent_result in result.results.items():
    print(f"\n=== RESULTADO DE {agent_name} ===")
    print(agent_result.result.message['content'][0]['text'])
    print(f"Status: {agent_result.status}")
    print(f"Execution time: {agent_result.execution_time}ms")


=== RESULTADO DE Culinary Coordinator ===
<thinking>I have handed off the task to the Ingredient Substitution Expert, Dietary Compliance Specialist, and Quality Assurance Chef to provide ingredient substitutions, confirm dietary compliance, and validate the final recipe. I will wait for their feedback to ensure the recipe meets all quality standards and provides an exceptional culinary experience. Once I receive their confirmation, I will compile the final recipe and provide it to the user.</thinking>

**Enhanced Recipe: Canelones de ternera XXL al Pedro Ximenez**

**Ingredients:**
- 500g of turkey or chicken (substituted for beef)
- 2 large zucchinis (instead of corn)
- 2 large carrots (instead of rice)
- 1 large onion (instead of garlic)
- 1 cup of vegetable broth (instead of milk)
- 1 cup of water (instead of eggs)
- 1 cup of tomato sauce (instead of gluten-containing sauces)
- 1 cup of spinach (instead of pepper)
- 10 sheets of rice paper (instead of gluten-containing pasta)

**In